## Reading data

In [2]:
import numpy as np
import os
import pandas as pd

In [3]:
PRJ = "/workspace/iceburger"
DATA = os.path.join(PRJ, "data/processed")

In [5]:
def image_normalization(x, percentile=1):
    """Normalize the image signal value by rescale data
    
    :param x: :class:`numpy.ndarray` of signal of dimension (height, width, 2)
    :param percentile: signal greater or less than the percentile will be capped
        as 1 and 0 respectively
    :returns: :class:`numpy.ndarray` of normalized 3 channel image with last
        channel totally black
    """
    vmax = np.percentile(x, 100 - percentile)
    vmin = np.percentile(x, percentile)
    x = (x - vmin) / (vmax - vmin)
    x[x > 1] = 1
    x[x < 0] = 0
    return np.concatenate([x, np.zeros(x.shape[:2] + (1,))], axis=-1)[np.newaxis, :, :, :]

In [6]:
def parse_json_data(json_filename):
    """Parse json data to generate trainable matrices
    
    :param json_filename: path to input json file
    :returns: a `tuple` of
        X: :class:`numpy.ndarray` of dimension (nb_samples, height, width, 3)
        X_angle: :class:`numpy.array` of dimension (nb_samples) of incidence
            angles
        y: :class:`numpy.array` of labels
    """
    df = pd.read_json(json_filename)
    dim = int(np.sqrt(len(df.band_1.iloc[0])))
    _X = np.concatenate([
        np.concatenate([np.array(r.band_1).reshape((dim, dim, 1)),
                        np.array(r.band_2).reshape((dim, dim, 1))],
                       axis=-1)[np.newaxis, :, :, :]
        for _, r in df.iterrows()], axis = 0)
    X = np.concatenate([image_normalization(x) for x in _X], axis=0)
    X_angle = df.inc_angle.values
    y = df.is_iceberg.values
    return (X, X_angle, y)

In [8]:
X_train, X_train_angle, y = parse_json_data(os.path.join(DATA, "train.json"))

In [9]:
print(X_train.shape)
print(X_train_angle.shape)
print(y.shape)

(1604, 75, 75, 3)
(1604,)
(1604,)


In [11]:
json=os.path.join(DATA, "train.json")
df_json = pd.read_json(json)
df_json.head()


,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878361, -27.15416, -28.668615, -29.537971...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920305, -14.920363, -12.66633...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


## Investigate incidence angle distribution

In [17]:
df_json.loc[df_json.inc_angle == "na", "inc_angle"] = np.NaN
df_json["inc_angle"] = df_json.inc_angle.astype(float)

In [20]:
df_json.inc_angle.isnull().mean()

0.082917705735660846

Around 8% of the incidence angle in training is null.

In [21]:
json_test=os.path.join(DATA, "test.json")
df_test = pd.read_json(json_test)
df_test.head()

,band_1,band_2,id,inc_angle
0,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",5941774d,34.966400
1,"[-26.0589694977, -26.0589694977, -26.058969497...","[-25.7542076111, -25.7542076111, -25.754207611...",4023181e,32.615072
2,"[-14.1410999298, -15.0642414093, -17.375520706...","[-14.745639801, -14.5904102325, -14.3626976013...",b20200e4,37.505433
3,"[-12.167478, -13.706167, -16.54837, -13.572674...","[-24.32222, -26.375538, -24.096739, -23.8769, ...",e7f018bb,34.473900
4,"[-23.3745937347, -26.0271816254, -28.121963501...","[-25.7223434448, -27.0115776062, -23.149162292...",4371c8c3,43.918874


In [28]:
df_test.inc_angle.isnull().sum()

0

No missing value in incidence angle in test set.

In [10]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [66]:
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 45)

In [67]:
genX1 = gen.flow(X_train, y,  batch_size=32, seed=666)
#genX2 = gen.flow(X_train_angle, y, batch_size=32, seed = 666)

In [69]:
X,Y =genX1.next()
X.shape
Y.shape

(32,)

In [15]:
w = 197
h = 197
XX = cv2.resize(X[0],(w,h))

In [16]:
XX.shape

(197, 197, 3)

In [17]:
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size= 32, seed=666)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

#Finally create out generator
gen_flow = gen_flow_for_one_input(X_train, y)

In [26]:
def exponential(args):
    X,angles,Y = args
    return (X[0]+j*X[1])*np.exp(-j*pi*angles/180)

## Resnet Model


In [38]:
import numpy as np
import pandas as pd
import os
import keras
from keras.optimizers import RMSprop
import shutil

from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout
from keras.models import Model

In [ ]:
def parse_json_data(json_filename):
    """Parse json data to generate trainable matrices

    :param json_filename: path to input json file
    :returns: a `tuple` of
        X: :class:`numpy.ndarray` of dimension (nb_samples, height, width, 3)
        X_angle: :class:`numpy.array` of dimension (nb_samples) of incidence
            angles
        y: :class:`numpy.array` of labels
    """
    df = pd.read_json(json_filename)
    dim = int(np.sqrt(len(df.band_1.iloc[0])))
    _X = np.concatenate([
        np.concatenate([np.array(r.band_1).reshape((dim, dim, 1)),
                        np.array(r.band_2).reshape((dim, dim, 1))],
                       axis=-1)[np.newaxis, :, :, :]
        for _, r in df.iterrows()], axis=0)
    X = np.concatenate([image_normalization(x) for x in _X], axis=0)
    X_angle = df.inc_angle.values
    y = df.is_iceberg.values
    if "set" in df.columns:
        subset = df["set"].values
    else:
        subset = np.array(["train"] * len(y))

    return (X, X_angle, y, subset)

In [73]:
X, X_angle, y, subset = parse_json_data(os.path.join(DATA, "train_valid.json"))
df = pd.read_json(os.path.join(DATA, "train_valid.json"))

In [ ]:
print(X.shape)
print(X_angle.shape)
print(y.shape)
print(subset.shape)
print(subset)
df.head()

In [81]:
X_train = np.array([cv2.resize(x,(w,h)) for x in X[subset=='train']])
X_angle_train = X_angle[subset=='train']
y_train = y[subset=='train']
X_valid = np.array([cv2.resize(x,(w,h)) for x in X[subset=='valid']])
X_angle_valid = X_angle[subset=='valid']
y_valid = y[subset=='valid']

In [83]:
X_train.shape

(1321, 197, 197, 3)

In [84]:
gen_train = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 45)

gen_valid = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 45)
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_train_for_one_input(X1, y):
    genX1 = gen_train.flow(X1, y, batch_size= 32, seed=666)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

def gen_flow_valid_for_one_input(X1, y):
    genX1 = gen_valid.flow(X1, y, batch_size= 32, seed=444)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]
#Finally create out generator
gen_flow_train = gen_flow_train_for_one_input(X_train, y_train)
gen_flow_valid = gen_flow_valid_for_one_input(X_valid, y_valid) 

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(197,197,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)
#for layer in base_model.layers:
#    layer.trainable = False
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping, ModelCheckpoint
epochs_to_wait_for_improve = 50
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
checkpoint_callback = ModelCheckpoint('BestKerasModelResNet50.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
model.summary()

In [85]:
#fit the model
model.fit_generator(gen_flow_train, validation_data= gen_flow_valid, validation_steps = int(np.ceil(len(X_valid)/32)), steps_per_epoch=int(np.ceil(len(X_train)/32)), epochs=500, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])
                    

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_12 (InputLayer)            (None, 197, 197, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_12 (ZeroPadding2D (None, 203, 203, 3)   0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 99, 99, 64)    9472                                         
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 99, 99, 64)    256                                          
___________________________________________________________________________________________

KeyboardInterrupt: 

## Convolutional Model

In [3]:
from __future__ import print_function
from __future__ import division
import numpy as np
import os
import logging
import argparse
import pandas as pd
import sys
import shutil

from keras import backend as K
from keras.engine.topology import get_source_inputs
from keras.layers import (Input, Activation, BatchNormalization, Conv2D,
                          Dense, Dropout, Flatten, GlobalAveragePooling2D,
                          GlobalMaxPooling2D, MaxPooling2D, Permute,
                          Reshape)
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

#from iceburger.io import parse_json_data
FORMAT =  '%(asctime)-15s %(name)-8s %(levelname)s %(message)s'
LOGNAME = 'iceburger-resnet50-train'

logging.basicConfig(format=FORMAT)
LOG = logging.getLogger(LOGNAME)
LOG.setLevel(logging.DEBUG)

PRJ = "/workspace/iceburger"
DATA = os.path.join(PRJ, "data/processed")


In [35]:
#: Number filters convolutional layers
NUM_CONV_FILTERS = 64

#: Size filters convolutional layers
CONV_FILTER_ROW = 3
CONV_FILTER_COL = 3

#: Dropout ratio
DROPOUT = 0.5

#: Regularization
L1L2R = 1E-3
L2R = 5E-3

batch_size=32
epochs=1000
train_steps=512
valid_steps=128
cb_early_stop=50
cb_reduce_lr=10
cb_reduce_lr_factor=0.5




In [6]:
def image_normalization(x, percentile=1):
    """Normalize the image signal value by rescale data
    
    :param x: :class:`numpy.ndarray` of signal of dimension (height, width, 2)
    :param percentile: signal greater or less than the percentile will be capped
        as 1 and 0 respectively
    :returns: :class:`numpy.ndarray` of normalized 3 channel image with last
        channel totally black
    """
    vmax = np.percentile(x, 100 - percentile)
    vmin = np.percentile(x, percentile)
    x = (x - vmin) / (vmax - vmin)
    x[x > 1] = 1
    x[x < 0] = 0
    return np.concatenate([x, np.zeros(x.shape[:2] + (1,))], axis=-1)[np.newaxis, :, :, :]

In [7]:
def parse_json_data(json_filename):
    """Parse json data to generate trainable matrices

    :param json_filename: path to input json file
    :returns: a `tuple` of
        X: :class:`numpy.ndarray` of dimension (nb_samples, height, width, 3)
        X_angle: :class:`numpy.array` of dimension (nb_samples) of incidence
            angles
        y: :class:`numpy.array` of labels
    """
    df = pd.read_json(json_filename)
    dim = int(np.sqrt(len(df.band_1.iloc[0])))
    _X = np.concatenate([
        np.concatenate([np.array(r.band_1).reshape((dim, dim, 1)),
                        np.array(r.band_2).reshape((dim, dim, 1))],
                       axis=-1)[np.newaxis, :, :, :]
        for _, r in df.iterrows()], axis=0)
    X = np.concatenate([image_normalization(x) for x in _X], axis=0)
    X_angle = df.inc_angle.values
    y = df.is_iceberg.values
    if "set" in df.columns:
        subset = df["set"].values
    else:
        subset = np.array(["train"] * len(y))

    return (X, X_angle, y, subset)

In [8]:
X, X_angle, y, subset = parse_json_data(os.path.join(DATA, "train_valid.json"))
df = pd.read_json(os.path.join(DATA, "train_valid.json"))

In [38]:
def get_callbacks(model_out_path):
    """
    Create list of callback functions for fitting step
    :param args: arguments as parsed by argparse module
    :returns: `list` of `keras.callbacks` classes
    """
    checkpoint_name= "{mn}-best_val_loss_weights.hdf5".format(mn="conv2d_model")
    callbacks = []
    #outpath = args.outpath

    # save best model so far
    callbacks.append(
        ModelCheckpoint(
            filepath=os.path.join(model_out_path, checkpoint_name),
            monitor="val_loss",
            verbose=1,
            save_best_only=True,
            save_weights_only=True
        )
    )
    if cb_early_stop:
        # stop training earlier if the model is not improving
        callbacks.append(
            EarlyStopping(
                monitor="val_loss",
                patience=cb_early_stop,
                verbose=1, mode='auto'
            )
        )
    if cb_reduce_lr:
        callbacks.append(
            ReduceLROnPlateau(
                monitor="val_loss",
                factor=cb_reduce_lr_factor,
                patience=10,
                min_lr=1e-8
            )
        )

    return callbacks, checkpoint_name



In [18]:
def compile_model(input_shape):
    """Build and compile model

    :param args: arguments as parsed by argparse module
    :returns: `keras.models.Model` of compiled model
    """
    model = Conv2D_Model(weights=None, include_top=True,
                         input_shape=input_shape)
    optimizer = SGD(lr = 0.0001, momentum = 0.9)
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [12]:
def conv2d_bn(x, filters, num_row, num_col, padding = 'same',
              strides = (1,1), name = 'None'):
    """Utility function to apply conv + BN.
    # Arguments
        x: input tensor.
        filters: filters in `Conv2D`.
        num_row: height of the convolution kernel.
        num_col: width of the convolution kernel.
        padding: padding mode in `Conv2D`.
        strides: strides in `Conv2D`.
        name: name of the ops; will become `name + '_conv'`
            for the convolution and `name + '_bn'` for the
            batch norm layer.
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`.
    """
    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x


In [42]:
def Conv2D_Model(include_top = True, weights=None, input_tensor = None,
         input_shape = None, pooling = None, classes = 2):
    """Instantiate the Conv architecture

    :param include_top: whether to include fully-connected layers at the top
        of the network
    :param weights: path to pretrained weights or `None`
    :param input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
        to use as image input for the model.
    :param input_shape: optional shape tuple, only to be specified if
        `include_top` is False
    :param pooling: optional pooling mode for feature extraction when
        `include_top` is False
    :param classes: optional number of classes to classify data into, only to
        be specified if `include_top` is True, and if no `weights` argument
        is specified.
    :returns: A :class:`keras.models.Model` instance
    """
    if input_tensor is None:
        image_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            image_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            image_input = input_tensor

    x = conv2d_bn(image_input, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn1")
    x = conv2d_bn(x, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn2")
    x = MaxPooling2D(pool_size=(2, 2), name="pool1")(x)

    x = conv2d_bn(x, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn3")
    x = conv2d_bn(x, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn4")
    x = MaxPooling2D(pool_size=(2, 2), name="pool2")(x)

    x = conv2d_bn(x, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn5")
    x = conv2d_bn(x, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn6")
    x = MaxPooling2D(pool_size=(2, 2), name="pool3")(x)

    x = conv2d_bn(x, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn7")
    x = conv2d_bn(x, NUM_CONV_FILTERS, CONV_FILTER_ROW, CONV_FILTER_COL, strides=(1,1),
                  name = "conv2d_bn8")
    x = MaxPooling2D(pool_size=(2, 2), name="pool4")(x)

    #x = Dropout(DROPOUT, name="drop1")(x)
    x = Flatten(name="flatten1")(x)
    if include_top:
        x = Dense(1, activation="sigmoid",
                  W_regularizer=l2(L2R),
                  b_regularizer=l2(L2R),
                  name="predictions")(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = image_input
    # Create model.
    model = Model(inputs, x, name='conv2d_model')
    if weights:
        model.load_weights(weights)

    return model


In [43]:
X_train = X[subset=='train']
X_angle_train = X_angle[subset=='train']
y_train = y[subset=='train']
X_valid = X[subset=='valid']
X_angle_valid = X_angle[subset=='valid']
y_valid = y[subset=='valid']

In [44]:
print(X_train.shape)
1728/64
print(X_valid.shape)

(1321, 75, 75, 3)
(283, 75, 75, 3)


In [45]:
model = compile_model(input_shape=(75,75,3))

/opt/conda/lib/python3.6/site-packages/Keras-2.0.0-py3.6.egg/keras/legacy/interfaces.py:86: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", name="predictions", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  '` call to the Keras 2 API: ' + signature)


In [46]:
gen_train = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 45)

gen_valid = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 45)


In [33]:
def gen_flow_for_one_input(X1, y):
        genX1 = gen_train.flow(X1, y, batch_size= 32, seed=666)
        while True:
            X1i = genX1.next()
            yield X1i[0], X1i[1]


In [47]:
gen_train_ = gen_train.flow(X_train, y_train, batch_size = 32, seed=666)
gen_valid_ = gen_valid.flow(X_valid, y_valid, batch_size = 32, seed=666)

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_bn1_conv (Conv2D)     (None, 75, 75, 64)        1728      
_________________________________________________________________
conv2d_bn1_bn (BatchNormaliz (None, 75, 75, 64)        192       
_________________________________________________________________
conv2d_bn1 (Activation)      (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_bn2_conv (Conv2D)     (None, 75, 75, 64)        36864     
_________________________________________________________________
conv2d_bn2_bn (BatchNormaliz (None, 75, 75, 64)        192       
_________________________________________________________________
conv2d_bn2 (Activation)      (None, 75, 75, 64)        0         
__________

In [49]:
model_out_path = os.path.join(os.path.abspath("./"),"checkpoints")
callbacks, checkpoint_model_name = get_callbacks(model_out_path)


In [ ]:
model.fit_generator(
        gen_train_,
        steps_per_epoch=train_steps,
        epochs=epochs, verbose=1,
        validation_data=gen_valid++_,
        validation_steps=valid_steps,
        callbacks=callbacks)


Epoch 1/1000
187/512 [=========>....................] - ETA: 1092s - loss: 0.6377 - acc: 0.6497